### Quickstart : Compare runs, choose a model, and deploy it to a REST API

topics convered :-
- Run a hyperparameter sweep on a training script
- compare the results of the runs in the MLflow UI
- choose the best run and register it as a model
- deploy the model to a REST API
- Build a container image suitable for deployment to a cloud platform

In [2]:
import keras
import pandas as pd
import numpy as np
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

import mlflow
from mlflow.models import infer_signature

In [3]:
# load the dataset
data = pd.read_csv(
    "https://raw.githubusercontent.com/mlflow/mlflow/master/tests/datasets/winequality-white.csv",
    sep=";",
)

data.head()

fixed acidity  volatile acidity  citric acid  residual sugar  chlorides  \
0            7.0              0.27         0.36            20.7      0.045   
1            6.3              0.30         0.34             1.6      0.049   
2            8.1              0.28         0.40             6.9      0.050   
3            7.2              0.23         0.32             8.5      0.058   
4            7.2              0.23         0.32             8.5      0.058   

   free sulfur dioxide  total sulfur dioxide  density    pH  sulphates  \
0                 45.0                 170.0   1.0010  3.00       0.45   
1                 14.0                 132.0   0.9940  3.30       0.49   
2                 30.0                  97.0   0.9951  3.26       0.44   
3                 47.0                 186.0   0.9956  3.19       0.40   
4                 47.0                 186.0   0.9956  3.19       0.40   

   alcohol  quality  
0      8.8        6  
1      9.5        6  
2     10.1        6  
3      9.9        6  
4      9.9        6

In [5]:
train_x = data.drop(['quality'], axis=1).values
train_y = data[['quality']].values.ravel()

# test dataset
test_x = data.drop(['quality'],axis=1).values
test_y = data[['quality']].values.ravel()

# splitting this train data into train and validation

train_x, valid_x, train_y, valid_y = train_test_split(train_x, train_y, test_size=0.20, random_state=42)

signature = infer_signature(train_x, train_y)

In [6]:
# ANN Model

def train_model(params, epochs, train_x, train_y, valid_x, valid_y, test_x, test_y):

    # define model architecture
    mean = np.mean(train_x, axis=0)
    var = np.var(train_x, axis=0)

    model = keras.Sequential(
        [
            keras.Input([train_x.shape[1]]),
            keras.layers.Normalization(mean=mean, variance=var),
            keras.layers.Dense(64, activation='relu'),
            keras.layers.Dense(1)
        ]
    )

    # compile the model
    model.compile(optimizer=keras.optimizers.SGD(
        learning_rate=params["lr"], momentum=params["momentum"]
    ),
    loss="mean_squared_error",
    metrics=[keras.metrics.RootMeanSquaredError()]
    )

    # train the ANN model with lr and momentum params with MLFLOW tracking
    with mlflow.start_run(nested=True):
        model.fit(train_x, train_y, validation_data=(valid_x, valid_y),epochs=epochs,batch_size=64)

        # evaluate the model
        eval_result = model.evaluate(valid_x, valid_y, batch_size=64)

        eval_rmse = eval_result[1]

        # log the parameters and results
        mlflow.log_params(params)
        mlflow.log_metric("eval_rmse", eval_rmse)

        # log the model
        mlflow.tensorflow.log_model(model, "model", signature=signature)

        return {"loss": eval_rmse, "status": STATUS_OK, "model": model}

In [7]:
def objective(params):
    # MLflow will track the parameters and results for each run
    result = train_model(
        params,
        epochs=3,
        train_x=train_x,
        train_y=train_y,
        valid_x=valid_x,
        valid_y=valid_y,
        test_x=test_x,
        test_y=test_y,
    )
    return result

In [8]:
space = {
    "lr" : hp.loguniform("lr", np.log(1e-5), np.log(1e-1)),
    "momentum" : hp.uniform("momentum", 0.0, 1.0)
}

In [10]:
mlflow.set_experiment("/wine-quality")
with mlflow.start_run():
    # conduct the hyperparameter search using hyperopt
    trials = Trials()
    best = fmin(
        fn= objective,
        space=space,
        algo=tpe.suggest,
        max_evals=4,
        trials=trials
    )

    # Fetch the details of the best run
    best_run = sorted(trials.results, key=lambda x: x["loss"])[0]

    # Log the best parameters, loss, and model
    mlflow.log_params(best)
    mlflow.log_metric("eval_rmse", best_run["loss"])
    mlflow.tensorflow.log_model(best_run["model"], "model", signature=signature)

    # Print out the best parameters and corresponding loss
    print(f"Best parameters: {best}")
    print(f"Best eval rmse: {best_run['loss']}") 

Epoch 1/3                                            

 1/62 ━━━━━━━━━━━━━━━━━━━━ 34s 570ms/step - loss: 37.7374 - root_mean_squared_error: 6.1431
35/62 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 36.7096 - root_mean_squared_error: 6.0588   
62/62 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 36.6560 - root_mean_squared_error: 6.0544 - val_loss: 35.8777 - val_root_mean_squared_error: 5.9898

Epoch 2/3                                            

 1/62 ━━━━━━━━━━━━━━━━━━━━ 2s 34ms/step - loss: 34.9742 - root_mean_squared_error: 5.9139
28/62 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 35.5266 - root_mean_squared_error: 5.9603 
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 35.3693 - root_mean_squared_error: 5.9472 - val_loss: 34.4240 - val_root_mean_squared_error: 5.8672

Epoch 3/3                                            

 1/62 ━━━━━━━━━━━━━━━━━━━━ 1s 32ms/step - loss: 34.2636 - root_mean_squared_error: 5.8535
25/62 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 34.3437 - root_mean_squared_error: 5.8

In [11]:
# Inferencing

from mlflow.models import validate_serving_input

model_uri = 'runs:/cf6f80fb3c9f4c458079c259c2346f15/model'

# the logged model does not contain an input example
# Manually generate a serving payload to verify your model prior to deployment
from mlflow.models import convert_input_example_to_serving_input

# Define INPUT_EXAMPLE via assignment with your own input example to the model
# A valid input example is a data instance suitable for pyfunc prediction
serving_payload = convert_input_example_to_serving_input(test_x)

# Validate the serving payload works on the model
validate_serving_input(model_uri, serving_payload)

c:\Users\pradeepd3\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


154/154 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step  


array([[6.33121  ],
       [4.0317307],
       [3.5139375],
       ...,
       [4.994628 ],
       [7.7092533],
       [6.5007963]], dtype=float32)

In [13]:
# another way
# Load model as a PyFuncModel
model_uri = 'runs:/cf6f80fb3c9f4c458079c259c2346f15/model'
loaded_model = mlflow.pyfunc.load_model(model_uri)

# Predict on a Pandas DataFrame
import pandas as pd
loaded_model.predict(pd.DataFrame(test_x))

154/154 ━━━━━━━━━━━━━━━━━━━━ 0s 792us/step


array([[6.33121  ],
       [4.0317307],
       [3.5139375],
       ...,
       [4.994628 ],
       [7.7092533],
       [6.5007963]], dtype=float32)

In [14]:
# Register in the model registry
mlflow.register_model(model_uri, "wine-quality")

Successfully registered model 'wine-quality'.
Created version '1' of model 'wine-quality'.


<ModelVersion: aliases=[], creation_timestamp=1746621153151, current_stage='None', description=None, last_updated_timestamp=1746621153151, name='wine-quality', run_id='cf6f80fb3c9f4c458079c259c2346f15', run_link=None, source='file:///c:/Users/Atharvb/Documents/Udemy_MLOps/mlflowstarter/2-DLMLFLOW/mlruns/651792501638623147/cf6f80fb3c9f4c458079c259c2346f15/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=1>